### Custom Metrics SDK

Use Domino’s Custom Model Monitoring Metrics SDK to define custom metrics and use them alongside out-of-the-box drift and model quality metrics that are monitored in Domino Model Monitor. With this SDK, you can register new metrics and define the logic to compute them. You can author this logic and evaluate it from within a Domino project.

For every model that you register for monitoring, you can select a registered metric, associate the data sources from which the metric is computed, and set up the execution environment to compute this metric on a periodic basis. You are notified by email when a metric behaves abnormally based on threshold definitions.

For end-to-end working code with a description of the workflow, see the /custom metrics example folder in the quick-start project


### Instantiate the client. Enter your DMM model ID:

In [1]:
import domino
import numpy as np
import pandas as pd
import datetime
import rfc3339

# Update your DMM Model ID
dmm_model_id = '657922ab9475d8d2f34562cf'
d = domino.Domino("dave_heinicke/Custom-Metric-Example")
metrics_client = d.custom_metrics_client()

metrics_client

### Log the custom metrics:

**modelMonitoringId:** ID of the monitored model to send metric alerts for

**metric**: Name of the metric to send alert for

**value:** Value of the metric

**timestamp:** Timezone is in UTC in ISO 8601 format.

**tags:** Custom metadata for metric represented as key-value string pairs

### Define your custom metric

Custom Metric for Iris Use Case: Hellinger Diatance

https://en.wikipedia.org/wiki/Hellinger_distance

In [2]:
def hellinger_distance(train, inference):
    
    # distance between training data and inference data
    # train is the ditribution of an input feature in the training data
    # inference is the dsitribution of a feature being sent to the model API
    
    n = min(len(train), len(inference))
    sum = 0.0
    
    for i in range(n):
        sum += (np.sqrt(train[i]) - np.sqrt(inference[i]))**2
        
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum)
    
    return result

#### Fetch Training set distribution for selected column

In [3]:
# Calculate the Metric
from domino.training_sets import TrainingSetClient, model

# Column we want to calculate metric for
drift_column_name = 'petal length (cm)'

# Print existing Training Sets in this Project

ts = TrainingSetClient.list_training_sets()
print(ts)


training_set = TrainingSetClient.get_training_set_version(
    training_set_name = "iris_python_multi_classification",
    number=1
    )

training_df = training_set.load_training_pandas()
train = training_df[drift_column_name]

print(train[:5])

[TrainingSet(name='iris_python_multi_classification', project_id='65788d6188931a7b02098d93', description=<training_set_api_client.types.Unset object at 0x7d6cadc52fa0>, meta={})]
0    6.1
1    4.0
2    4.2
3    6.9
4    4.7
Name: petal length (cm), dtype: float64


#### Fetch Inference data

In [4]:
scoring_data = pd.read_csv('/domino/datasets/local/Custom-Metric-Example/iris_scoring_data_2023-12-18.csv')

inference = scoring_data[drift_column_name]

#### Calculate your metric

In [5]:
hellinger_distance = hellinger_distance(train, inference)
print('Hellinger distance between scoring and traiing data is: {}'.format(str(round(hellinger_distance, 3))))

Hellinger distance between scoring and traiing data is: 6.623


#### Log your metric with Model Monitoring

In [11]:
# Retrieve the stored metrics for the last 3 years 
startDate = datetime.datetime.today() - datetime.timedelta(days=365*3)
startDate = rfc3339.rfc3339(startDate)
endDate = rfc3339.rfc3339(datetime.datetime.today())

# Retrieve the metrics
try:
    res = metrics_client.read_metrics(dmm_model_id, "hellinger_distance", startDate, endDate)
    
except Exception as err:
    logging.error("Unable to fetch metrics")
    raise err
    
print(res)

{'metadata': {'requestId': 'b39985d7-5ef5-4277-a016-545f32ad3c11', 'notices': ()}, 'metricValues': [{'timestamp': '2023-12-20T02:50:51Z', 'value': 6.622582099042992, 'tags': {'Column': 'petal length (cm)'}}, {'timestamp': '2023-12-20T14:33:27Z', 'value': 6.622582099042992, 'tags': {'Column': 'petal length (cm)'}}]}


In [7]:
# timestamp = "2023-12-17T00:00:00Z"
timestamp = rfc3339.rfc3339(datetime.datetime.now()) # datetime.datetime.now().isoformat()

print(dmm_model_id)
print(hellinger_distance)
print(timestamp)
print(drift_column_name)

metrics_client.log_metric(dmm_model_id, "hellinger_distance", hellinger_distance, timestamp, { "Column" : drift_column_name})

# metrics_client.log_metrics([
# { "modelMonitoringId" : dmm_model_id, "metric" : "accuracy", "value" : 7.1234,
# "timestamp" : "2022-10-08T00:00:00Z",
# "tags" : { "example_tag1" : "value1", "example_tag2" : "value2" }
# ]
# },
# { "modelMonitoringId" : dmm_model_id, "metric" : "other_metric", "value" : 8.4567,
# "timestamp" : "2022-10-09T00:00:00Z" }
# ])


657922ab9475d8d2f34562cf
6.622582099042992
2023-12-20T14:33:27+00:00
petal length (cm)


#### Send a custom metrics alert:

**modelMonitoringId:** ID of the monitored model for which to send metric alerts.

**metric:** Name of the metric for which to send the alert.

**value:** Value of the metric.

**condition:** Target range for the metric defined by lower and upper limit bounds.
The following are potential values for the condition argument:

    metrics_client.LESS_THAN = "lessThan"

    metrics_client.LESS_THAN_EQUAL = "lessThanEqual"

    metrics_client.GREATER_THAN = "greaterThan"

    metrics_client.GREATER_THAN_EQUAL = "greaterThanEqual"

    metrics_client.BETWEEN = "between"

**lower_limit:** The lower limit for the condition.

**upper_limit:** The upper limit for the condition.

**description:** Optional message included in the alert.

In [9]:
metrics_client.trigger_alert(dmm_model_id, 
                             "hellinger_distance", 
                             hellinger_distance, 
                             condition = metrics_client.GREATER_THAN, 
                             lower_limit=6.0,
                             upper_limit=8.0,
                             description = "Hellinger distance exceeds 6.0." )